# מנגנוני קשב ומודלים של טרנספורמרים

חיסרון מרכזי של רשתות חוזרות (Recurrent Networks) הוא שכל המילים ברצף משפיעות באותה מידה על התוצאה. הדבר גורם לביצועים תת-אופטימליים במודלים סטנדרטיים של LSTM עבור משימות רצף לרצף, כמו זיהוי ישויות בשם ותרגום מכונה. במציאות, למילים מסוימות ברצף הקלט יש השפעה רבה יותר על הפלט הרציף מאשר לאחרות.

נבחן מודל רצף לרצף, כמו תרגום מכונה. מודל כזה מיושם באמצעות שתי רשתות חוזרות, כאשר רשת אחת (**מקודד**) דוחסת את רצף הקלט למצב מוסתר, ורשת אחרת, **מפענח**, פורסת את המצב המוסתר הזה לתוצאה מתורגמת. הבעיה בגישה זו היא שהמצב הסופי של הרשת מתקשה לזכור את תחילת המשפט, מה שגורם לאיכות ירודה של המודל במשפטים ארוכים.

**מנגנוני קשב** מספקים דרך לשקלל את ההשפעה ההקשרית של כל וקטור קלט על כל תחזית פלט של ה-RNN. הדבר מיושם על ידי יצירת קיצורי דרך בין המצבים הביניים של ה-RNN של הקלט לבין ה-RNN של הפלט. כך, בעת יצירת סמל פלט $y_t$, ניקח בחשבון את כל המצבים המוסתרים של הקלט $h_i$, עם מקדמי משקל שונים $\alpha_{t,i}$.

![תמונה המציגה מודל מקודד/מפענח עם שכבת קשב אדיטיבית](../../../../../translated_images/he/encoder-decoder-attention.7a726296894fb567.webp)
*מודל מקודד-מפענח עם מנגנון קשב אדיטיבי מתוך [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), מצוטט מתוך [פוסט הבלוג הזה](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

מטריצת הקשב $\{\alpha_{i,j}\}$ מייצגת את המידה שבה מילים מסוימות בקלט משפיעות על יצירת מילה מסוימת ברצף הפלט. להלן דוגמה למטריצה כזו:

![תמונה המציגה יישור דוגמה שנמצא על ידי RNNsearch-50, מתוך Bahdanau - arviz.org](../../../../../translated_images/he/bahdanau-fig3.09ba2d37f202a6af.webp)

*תמונה מתוך [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (איור 3)*

מנגנוני קשב אחראים לחלק ניכר מהמצב הנוכחי או הקרוב למצב הנוכחי של האמנות בתחום עיבוד שפה טבעית. עם זאת, הוספת קשב מגדילה משמעותית את מספר הפרמטרים במודל, מה שהוביל לבעיות סקלאביליות עם RNNs. מגבלה מרכזית בסקלאביליות של RNNs היא שהאופי החוזר של המודלים מקשה על ביצוע אצווה (batching) והקבלה במקביל. ב-RNN, כל אלמנט ברצף צריך להיות מעובד בסדר רציף, מה שאומר שלא ניתן להקביל בקלות את התהליך.

האימוץ של מנגנוני קשב בשילוב עם מגבלה זו הוביל ליצירת מודלים של טרנספורמרים, שהם כיום המצב המתקדם ביותר (State of the Art) שאנו מכירים ומשתמשים בהם, כמו BERT ו-OpenGPT3.

## מודלים של טרנספורמרים

במקום להעביר את ההקשר של כל תחזית קודמת לשלב ההערכה הבא, **מודלים של טרנספורמרים** משתמשים ב**קידודים מיקום** וב**קשב** כדי ללכוד את ההקשר של קלט נתון בתוך חלון טקסט מסוים. התמונה למטה מראה כיצד קידודי מיקום עם קשב יכולים ללכוד הקשר בתוך חלון נתון.

![GIF מונפש המציג כיצד מתבצעות ההערכות במודלים של טרנספורמרים.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

מכיוון שכל מיקום קלט ממופה באופן עצמאי לכל מיקום פלט, טרנספורמרים יכולים להקביל טוב יותר מ-RNNs, מה שמאפשר יצירת מודלים לשוניים גדולים ומרשימים יותר. כל ראש קשב יכול לשמש ללמידת יחסים שונים בין מילים, מה שמשפר משימות עיבוד שפה טבעית.

## בניית מודל טרנספורמר פשוט

Keras אינה כוללת שכבת טרנספורמר מובנית, אך ניתן לבנות אחת בעצמנו. כמו קודם, נתמקד בסיווג טקסט של מערך הנתונים AG News, אך כדאי לציין שמודלים של טרנספורמרים מציגים את התוצאות הטובות ביותר במשימות NLP מורכבות יותר.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

שכבות חדשות ב-Keras צריכות לרשת את מחלקת `Layer`, ולממש את המתודה `call`. נתחיל עם שכבת **Positional Embedding**. נשתמש [בקוד מתוך תיעוד רשמי של Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). נניח שאנו מרפדים את כל רצפי הקלט לאורך `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

שכבה זו מורכבת משתי שכבות `Embedding`: אחת להטמעת טוקנים (בדרך שדנו בה קודם) ואחת להטמעת מיקומי טוקנים. מיקומי הטוקנים נוצרים כרצף של מספרים טבעיים מ-0 עד `maxlen` באמצעות `tf.range`, ולאחר מכן מועברים דרך שכבת ההטמעה. שני וקטורי ההטמעה המתקבלים מחוברים יחד, ומייצרים ייצוג מוטמע-מיקומית של הקלט בצורת `maxlen`$\times$`embed_dim`.

עכשיו, בואו נממש את בלוק הטרנספורמר. הוא יקבל את הפלט של שכבת ההטמעה שהוגדרה קודם:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

עכשיו, אנחנו מוכנים להגדיר את מודל הטרנספורמר המלא:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## מודלים של BERT Transformer

**BERT** (Bidirectional Encoder Representations from Transformers) הוא רשת טרנספורמר רב שכבתית גדולה מאוד עם 12 שכבות עבור *BERT-base*, ו-24 עבור *BERT-large*. המודל עובר תחילה אימון מוקדם על מאגר נתונים גדול של טקסט (ויקיפדיה + ספרים) באמצעות אימון לא מפוקח (ניבוי מילים מוסתרות במשפט). במהלך האימון המוקדם, המודל סופג רמה משמעותית של הבנת שפה, שניתן לאחר מכן לנצל עם מערכי נתונים אחרים באמצעות כיוונון עדין. תהליך זה נקרא **למידה מעוברת**.

![תמונה מתוך http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/he/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

ישנם וריאציות רבות של ארכיטקטורות טרנספורמר, כולל BERT, DistilBERT, BigBird, OpenGPT3 ועוד, שניתן לבצע עליהן כיוונון עדין.

בואו נראה כיצד ניתן להשתמש במודל BERT שעבר אימון מוקדם כדי לפתור את בעיית סיווג הרצפים המסורתית שלנו. נשתמש ברעיון ובחלק מהקוד מתוך [התיעוד הרשמי](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

כדי לטעון מודלים שעברו אימון מוקדם, נשתמש ב-**Tensorflow hub**. ראשית, נטען את הוקטורייזר הספציפי ל-BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

חשוב להשתמש באותו וקטורייזר שבו הרשת המקורית אומנה. בנוסף, הוקטורייזר של BERT מחזיר שלושה רכיבים:
* `input_word_ids`, שהוא רצף של מספרי טוקנים עבור המשפט הקלט
* `input_mask`, שמראה איזה חלק מהרצף מכיל קלט אמיתי ואיזה חלק הוא ריפוד. זה דומה למסכה שמיוצרת על ידי השכבה `Masking`
* `input_type_ids` משמש למשימות של מודל שפה, ומאפשר לציין שני משפטי קלט ברצף אחד.

לאחר מכן, נוכל ליצור מופע של מחלץ התכונות של BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


אז, שכבת BERT מחזירה מספר תוצאות שימושיות:
* `pooled_output` הוא תוצאה של ממוצע כל הטוקנים ברצף. ניתן לראות אותו כייצוג סמנטי חכם של כל הרשת. הוא שווה ערך לפלט של שכבת `GlobalAveragePooling1D` במודל הקודם שלנו.
* `sequence_output` הוא הפלט של שכבת הטרנספורמר האחרונה (מקביל לפלט של `TransformerBlock` במודל שלנו למעלה).
* `encoder_outputs` הם הפלטים של כל שכבות הטרנספורמר. מכיוון שטענו מודל BERT עם 4 שכבות (כפי שניתן לנחש מהשם, שמכיל `4_H`), יש לו 4 טנסורים. האחרון מביניהם זהה ל-`sequence_output`.

כעת נגדיר את מודל הסיווג מקצה לקצה. נשתמש ב-*הגדרת מודל פונקציונלית*, שבה נגדיר את הקלט של המודל, ולאחר מכן נספק סדרת ביטויים לחישוב הפלט שלו. בנוסף, נקבע את משקלי מודל ה-BERT כלא-ניתנים לאימון, ונאמן רק את מסווג הסיום:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


למרות שיש מעט פרמטרים שניתן לאמן, התהליך די איטי, מכיוון שמחלץ התכונות של BERT הוא כבד מבחינה חישובית. נראה שלא הצלחנו להגיע לדיוק סביר, או בגלל חוסר באימון, או בגלל חוסר בפרמטרים של המודל.

בואו ננסה לשחרר את המשקלים של BERT ולאמן גם אותם. זה דורש קצב למידה מאוד קטן, וגם אסטרטגיית אימון זהירה יותר עם **חימום מוקדם**, תוך שימוש בממטב **AdamW**. נשתמש בחבילה `tf-models-official` כדי ליצור את הממטב:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

כפי שאתם יכולים לראות, תהליך האימון מתקדם לאט - אבל ייתכן שתרצו להתנסות ולאמן את המודל לכמה אפוקים (5-10) ולבדוק אם תוכלו להשיג את התוצאה הטובה ביותר בהשוואה לגישות שהשתמשנו בהן קודם.

## ספריית Huggingface Transformers

דרך נוספת נפוצה (וקצת פשוטה יותר) להשתמש במודלים של Transformer היא [חבילת HuggingFace](https://github.com/huggingface/), שמספקת אבני בניין פשוטות למשימות שונות בתחום עיבוד שפה טבעית (NLP). החבילה זמינה גם עבור Tensorflow וגם עבור PyTorch, מסגרת רשתות עצביות פופולרית נוספת.

> **Note**: אם אינכם מעוניינים לראות כיצד ספריית Transformers עובדת - תוכלו לדלג לסוף המחברת, מכיוון שלא תראו שום דבר שונה באופן מהותי ממה שעשינו קודם. אנו נחזור על אותם שלבים של אימון מודל BERT באמצעות ספרייה שונה ומודל גדול בהרבה. לכן, התהליך כולל אימון די ממושך, כך שייתכן שתרצו פשוט לעבור על הקוד.

בואו נראה כיצד ניתן לפתור את הבעיה שלנו באמצעות [Huggingface Transformers](http://huggingface.co).


הדבר הראשון שצריך לעשות הוא לבחור את המודל שבו נשתמש. בנוסף לכמה מודלים מובנים, Huggingface מכיל [מאגר מודלים מקוון](https://huggingface.co/models), שבו ניתן למצוא הרבה מודלים מוכנים מראש שפותחו על ידי הקהילה. כל אחד מהמודלים הללו ניתן לטעון ולהשתמש בו פשוט על ידי מתן שם המודל. כל הקבצים הבינאריים הנדרשים עבור המודל יורדו באופן אוטומטי.

במקרים מסוימים תצטרכו לטעון מודלים משלכם, ובמקרה כזה תוכלו לציין את הספרייה שמכילה את כל הקבצים הרלוונטיים, כולל פרמטרים עבור ה-tokenizer, קובץ `config.json` עם פרמטרי המודל, משקלים בינאריים, ועוד.

מתוך שם המודל, ניתן ליצור גם את המודל וגם את ה-tokenizer. בואו נתחיל עם ה-tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

האובייקט `tokenizer` מכיל את הפונקציה `encode` שניתן להשתמש בה ישירות לקידוד טקסט:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

אנחנו יכולים גם להשתמש במפענח כדי לקודד רצף בצורה שמתאימה להעברה למודל, כלומר כולל שדות `token_ids`, `input_mask` וכו'. אנחנו יכולים גם לציין שאנחנו רוצים טנסורים של Tensorflow על ידי מתן ארגומנט `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

במקרה שלנו, נשתמש במודל BERT מאומן מראש שנקרא `bert-base-uncased`. *Uncased* מציין שהמודל אינו רגיש לאותיות גדולות או קטנות.

כאשר אנו מאמנים את המודל, עלינו לספק רצף מחולק לטוקנים כקלט, ולכן נעצב צינור עיבוד נתונים. מכיוון ש-`tokenizer.encode` היא פונקציה ב-Python, נשתמש באותה גישה כמו ביחידה הקודמת ונקרא לה באמצעות `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

עכשיו אנחנו יכולים לטעון את המודל עצמו באמצעות החבילה `BertForSequenceClassification`. זה מבטיח שלמודל שלנו כבר יש את הארכיטקטורה הנדרשת לסיווג, כולל המסווג הסופי. תראו הודעת אזהרה שמציינת שהמשקלים של המסווג הסופי לא מאותחלים, והמודל ידרוש אימון מוקדם - זה לגמרי בסדר, כי זה בדיוק מה שאנחנו עומדים לעשות!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


כפי שניתן לראות מ-`summary()`, המודל מכיל כמעט 110 מיליון פרמטרים! כנראה שאם נרצה משימת סיווג פשוטה על מערך נתונים קטן יחסית, לא נרצה לאמן את שכבת הבסיס של BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


עכשיו אנחנו מוכנים להתחיל את האימון!

> **הערה**: אימון מודל BERT בקנה מידה מלא יכול להיות מאוד זמן רב! לכן, נאמן אותו רק עבור 32 האצוות הראשונות. זה רק כדי להראות איך תהליך האימון של המודל מוגדר. אם אתם מעוניינים לנסות אימון בקנה מידה מלא - פשוט הסירו את הפרמטרים `steps_per_epoch` ו-`validation_steps`, והכינו את עצמכם להמתנה!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


אם תגדילו את מספר האיטרציות ותמתינו מספיק זמן, ותאמנו במשך מספר אפוקים, תוכלו לצפות שמודל הסיווג של BERT ייתן לנו את הדיוק הטוב ביותר! זאת מכיוון ש-BERT כבר מבין היטב את מבנה השפה, וכל מה שנדרש הוא לכוונן את מסווג הסיום. עם זאת, מכיוון ש-BERT הוא מודל גדול, כל תהליך האימון לוקח זמן רב ודורש כוח חישובי משמעותי! (כרטיס גרפי, ורצוי יותר מאחד).

> **הערה:** בדוגמה שלנו, השתמשנו באחד מהמודלים הקטנים ביותר של BERT שהוכנו מראש. קיימים מודלים גדולים יותר שסביר להניח שיניבו תוצאות טובות יותר.


## נקודות חשובות

ביחידה זו, ראינו ארכיטקטורות מודלים עדכניות המבוססות על **טרנספורמרים**. יישמנו אותן במשימת סיווג הטקסט שלנו, אך באופן דומה, ניתן להשתמש במודלים של BERT גם לחילוץ ישויות, מענה על שאלות ומשימות עיבוד שפה טבעית (NLP) נוספות.

מודלים מבוססי טרנספורמרים מייצגים את חוד החנית הנוכחי בתחום ה-NLP, וברוב המקרים הם צריכים להיות הפתרון הראשון שבו תתחילו להתנסות כאשר אתם מיישמים פתרונות מותאמים אישית בתחום זה. עם זאת, הבנה של העקרונות הבסיסיים של רשתות עצביות חוזרות (RNN), שנדונו במודול זה, היא חשובה ביותר אם ברצונכם לבנות מודלים עצביים מתקדמים.



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
